# Structurer et explorer des données textuelles

Notebook Introduction au traitement du langage naturel - 15/05/2025 - Émilien Schultz

## Les données

Open Alex

## Les bibliothèques

- `pandas` pour la manipulation de données
- `nltk` pour le traitement de texte
- `matplotlib` pour la visualisation
- `scikit-learn` pour le traitement de texte et la modélisation


In [1]:
#%pip install pandas nltk scikit-learn matplotlib

## Nettoyer les données (préprocessing)

- Supprimer les doublons
- Supprimer les lignes vides
- Convertir en minuscules
- Garder uniquement de l'anglais

In [2]:
import pandas as pd
df = pd.read_csv("../data/CSS_exact_openalex.csv")

df["abstract"].isna().sum()
df = df[~df["abstract"].isna()]

df["texte"] = df["title"] + " " + df["abstract"]

## Analyse à l'échelle des mots

### Chercher la présence d'un mot

Les bases de la fouille de données. Quels sont les questions qui parlent d'intelligence artificielle ?

In [3]:
filtre = df["texte"].str.contains("algorithm")
filtre.sum()

77

Chercher un contexte d'un mot avec uen expression régulière

In [4]:
import re
re.findall(".{5}algorithm.{5}", df[filtre].loc[304, "texte"])

['work algorithms, wh',
 'bust algorithms for',
 'n an algorithm is p',
 " The algorithm's go"]

In [5]:
(df["texte"].str.lower()
            .str.contains("artificial intelligence")
            .sum())

34

Et si on cherche plusieurs termes ?

In [6]:
termes = ["AI", "algorithm"]

(df["texte"].str.lower()
            .str.contains("|".join(termes))
            .sum())

77

Faire une recherche sur toutes les variables possibles de l'IA

- intelligence artificelle
- algorithme
- AI
- ...

### Tokenisation

Découper un texte

#### Utiliser les regex

In [7]:
import re
word_pattern = r"\w+"
tokens = re.findall(word_pattern, "Ceci est un test")
tokens

['Ceci', 'est', 'un', 'test']

In [8]:
df["texte"].apply(lambda x: re.findall(r"\w+",x.lower()))

0       [computational, social, science, 14, 0642, 033...
1       [manifesto, of, computational, social, science...
3       [computational, social, science, and, sociolog...
7       [can, large, language, models, transform, comp...
9       [on, agent, based, modeling, and, computationa...
                              ...                        
1436    [area, studies, and, the, challenges, of, crea...
1439    [we, talk, data, we, do, data, welcome, to, th...
1441    [prediction, machines, my, essay, has, several...
1447    [index, citation, 2020, index, härtel, c, e, j...
1448    [index, citation, 2023, index, lytras, m, d, h...
Name: texte, Length: 690, dtype: object

#### Utiliser une première bibliothèque : `nltk`

In [12]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

word_tokenize("Ceci est un test")

[nltk_data] Downloading package punkt_tab to /Users/leo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['Ceci', 'est', 'un', 'test']

In [ ]:
df["texte"].apply(word_tokenize)

0       [Computational, Social, Science, 14,0642,033Me...
1       [Manifesto, of, computational, social, science...
3       [Computational, Social, Science, and, Sociolog...
7       [Can, Large, Language, Models, Transform, Comp...
9       [On, agent-based, modeling, and, computational...
                              ...                        
1436    [Area, Studies, and, the, Challenges, of, Crea...
1439    [We, talk, data, ., We, do, data, ., Welcome, ...
1441    [Prediction, Machines, My, essay, has, several...
1447    [Index, Citation, (, 2020, ), ,, ``, Index, ''...
1448    [Index, Citation, (, 2023, ), ,, ``, Index, ''...
Name: texte, Length: 690, dtype: object

### Quels sont les termes les plus fréquents ?

In [ ]:
from collections import Counter

In [ ]:
compteur = Counter([j for i in list(df["texte"].apply(word_tokenize)) for j in i])

In [ ]:
compteur.most_common(20)

[(',', 9699),
 ('the', 6493),
 ('of', 6025),
 ('and', 5878),
 ('.', 5125),
 ('to', 3420),
 ('in', 3088),
 ('a', 2415),
 ('social', 1953),
 ('for', 1601),
 (')', 1491),
 ('(', 1471),
 ('on', 1340),
 ('that', 1282),
 ('data', 1180),
 ('is', 1061),
 ('as', 942),
 ('science', 912),
 ('with', 901),
 ('computational', 887)]

### Quelles sont les expressions qui reviennent le plus souvent ?

Utilisons les bigrammes et les trigrammes

In [ ]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def generate_bigrams_nltk(text):
    tokens = word_tokenize(text.lower())
    bigrams = list(ngrams(tokens, 2))
    return bigrams

#generate_bigrams_nltk(df["texte_net"].iloc[0])

In [ ]:
word_tokenize("ici-même, y'a-t'il")

['ici-même', ',', 'y', "'", "a-t'il"]

#### Enlever les stop words

In [ ]:
nltk.download("stopwords")

from nltk.corpus import stopwords

english_stopwords = list(set(stopwords.words("english")))
english_stopwords[0:10]


def generate_bigrams_nltk(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in english_stopwords]
    bigrams = list(ngrams(filtered_tokens, 2))
    return bigrams

#generate_bigrams_nltk(df["texte_net"].iloc[0])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emilien/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Count bigrams:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=french_stopwords, ngram_range=(2, 2), max_features=300)
bigrammes = (
    pd.DataFrame(
        vectorizer.fit_transform(df["texte"]).toarray(),
        columns=vectorizer.get_feature_names_out(),
    )
    .T.sum(axis=1)
    .sort_values(ascending=False)
)
bigrammes

social science          882
computational social    872
social media            350
social sciences         189
big data                169
                       ... 
socioeconomic status      9
comparative analysis      9
strength social           9
age gender                9
models social             9
Length: 300, dtype: int64

## Représenter les textes

### Présence de mots

In [ ]:
df["dim1"] = df["texte"].str.contains("AI")
df["dim2"] = df["texte"].str.contains("science")
df[["dim1","dim2"]].replace({True:1,False:0}).head()

/var/folders/f9/d2d_05ws5gncmml0fx0c00kw0000gp/T/ipykernel_77900/386222696.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[["dim1","dim2"]].replace({True:1,False:0}).head()


,dim1,dim2
0,0,0
1,0,1
3,0,1
7,0,1
9,0,1


### Vecteur brut : Document term matrix / tableau

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# créer mon object de ML
vectorizer = CountVectorizer(stop_words=english_stopwords, 
                             ngram_range=(1, 1), 
                             max_features=800)

# appliquer sur les données
X = vectorizer.fit_transform(df["texte"])
X = pd.DataFrame(X.toarray(),columns=list(vectorizer.get_feature_names_out()))

In [ ]:
# X.iloc[2]

### Une version un peu plus avancée

- Term Frequency-Inverse Document Frequency
    - Amélioration du DTM
- Approche souvent utilisée pour mettre en valeur les mots les plus spécifiques
- `Scikit-learn` a `TfidfVectorizer`

$$\text{TF-IDF}(t, d, D) = \left( \frac{f_{t,d}}{n_d} \right) \times \log \left(\frac{N}{\text{df}_t} \right)
$$

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# créer un objet
vectorizer = TfidfVectorizer(stop_words=english_stopwords, 
                             ngram_range=(1, 1), 
                             max_features=800)

# applique 
X = vectorizer.fit_transform(df["texte"])

# mettre en forme
X = pd.DataFrame(X.toarray(),columns=list(vectorizer.get_feature_names_out()))
X.loc[100].sort_values()

10               0.000000
politics         0.000000
popular          0.000000
popularity       0.000000
population       0.000000
                   ...   
even             0.000000
era              0.000000
social           0.559948
computational    0.560759
science          0.609924
Name: 100, Length: 800, dtype: float64

In [ ]:
len(vectorizer.get_feature_names_out())

800

Faire la matrice TF-IDF, identifier les mots qui ont le score le plus important

## Distance entre deux textes

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

X = vectorizer.fit_transform(df["texte"])
cosine_similarity(X[0], X[4])

array([[0.02197339]])

In [ ]:
distances = pd.DataFrame(pairwise_distances(X, metric="cosine"))

In [ ]:
distances[10].idxmax()

## Application : Faire un nuage de mots avec WordCloud

Un coup d'oeil à la [documentation](https://amueller.github.io/word_cloud/)